## Calculating the Tropopause from Sounding Data

The GitHub issue we will be addressing is calculating the height of the tropopause using data from soundings. This calculation is currently unavailable from the Metpy reference guide, and its addition would benefit many researchers. 

The tropopause is the boundary that exists between the lowest levels of the atmosphere, the troposphere and the stratosphere. This boundary is usually identified by a change of lapse rate and an increase in stability as the boundary is crossed. Rather than existing at a single height above the surface, the height of the tropopause varies from 15 to 20 km in the Tropics to 6 km in polar areas (American Meteorological Society, 2020). Additionally, there exist instances of discontinuity in the tropopause where its levels overlap in a series of “leaves”. This multiple tropopause condition occurs most frequently over areas of the troposphere where there are large contrasts of horizontal temperature. This existence of multiple tropopauses or “leaves” complicates the computation of its height (American Meteorological Society, 2020). 

The understanding of the chemical and radiative interaction between the Troposphere and the Stratosphere is critical to evaluate the effects of climate change (Santer et al., 2003). The increase in altitude of the Tropopause is not only a sign of warming in the Troposphere, but it has also been related with the strengthening of the subtropical jets (Manney and Hegglin, 2018). An accurate calculation of the Tropopause’s altitude is important to assess the warming in the long term, and identify the exchange of water vapor (Mote et al., 1996) and ozone (Bethan et al., 1996) between the upper Troposphere and the lower Stratosphere as an indicator of greenhouse gases transportation.
 
There are several ways to calculate the altitude of the Tropopause. The majority of the studies focus on Thermodynamics: The reduction of the lapse rate to 2 K or less per kilometer was the criteria established by the World Meteorological Organization in 1992, and has been proved to be globally reliable and matches with the chemical and stability transitions, and the cloud top occurrence (Pan, 2011). Nevertheless, there are some exceptions when this criteria does not apply: During Rossby waves breaking events when there is a transport of air between the Upper Troposphere and the Extratropical Lower Stratosphere (Homeyer and Bowman, 2013); when the Lower Stratosphere is weakly (and abnormally) stable, like in the Antarctic during the winter. 

Another approach is to find the sharp edge between the unstable Troposphere and the stable layered Stratosphere. However, this technique to find the Tropopause fails in the deep Tropics and zones with high cyclogenesis where we can find phenomena such as the double (and even multiple) Tropopause (Añil et al., 2008). The coldest temperature is another technique that has been proved it is useful just close to the Equator. The dynamical Tropopause is defined by a value of 6 pvu Potential Vorticity by Homeyer et al. in 2011 for a study of intrusion of Ozone and other chemicals from the Troposphere into the Stratosphere; however, it is just useful for locations to the Poles proximity and adjacent to the Subtropical jets, since PV is zero at the Equator. Seasonal and Climatological variations impairs the use of the Ozone profiles as a 100% reliable technique to find the Tropopause in all contexts. A nice summary of the techniques can be found at Xian and Homeyer, 2019.

The NCAR Command Language (NCL) has a version of code for calculating the tropopause, first developed by Brunner et al. (2009) on Fortran. This code uses the 1992 WMO criteria based on temperature radiosonde data, and we will use this as a starting point for writing code to calculate the tropopause. This source code is written in Fortran so our process will involve converting it to Python. Changes may exist between this source code and our selected method, so if necessary we will adapt the NCL code so it best fits the method of calculation we select.

When our code for calculating the tropopause is completed we will run test cases using past sounding data in both the method we have selected and the original NCL method. We will obtain sounding data from the University of Wyoming archive to test both methods. Our goal with this comparison is to verify that our method accurately calculates the height of the tropopause and determine any discrepancies that exist between our method and the NCL method.

## Description of NCAR's Implementation of Tropopause Height Calculations

https://github.com/NCAR/ncl/blob/develop/ni/src/lib/nfpfort/stattrop_dp.f

The above source code was referenced in the original github MetPy issue #1324. The source code is written in Fortran and implements WMO's (1992) definition of the tropopause height:

```The first tropopause is defined as the lowest level at which the lapse rate decreases to 2 deg K per kilometer or less, provided also the average lapse rate between this level and all higher levels within 2 kilometers does not exceed 2 deg K.```

The implementation takes sounding data (Temperature and Pressure) and uses it to calculate the lapse rate $\frac{dT}{dz}$. This calculation is achieved by applying the hydrostatic approximation and the ideal gas law to say:

$-\frac{dT}{dz} = C \frac{d(lnT)}{d(lnp)}$

Where $C$ is the constant $g/R \times 1000 \frac{m}{km}$. Here, $g$ is the acceleration due to gravity, $R$ is the ideal gas constant, and the factor of $1000$ is simply to convert the lapse rate to $\frac{K}{km}$.

The implementation then proceeds as follows:

-- Use a two point forward difference to approximate $\frac{dT}{dz}$ for all heights

-- Starting from the surface, loop through the calculated lapse rates and check the following conditions:

  1) Is the lapse rate less than $2 \frac{K}{km}$ ?

  2) If so, linearly interpolate a tropopause height from pressure, find the pressure $2 km$ above this using the hypsometric equation, and check whether the arithmetic average of the lapse rate between these two pressures is less than $2 \frac{K}{km}$

-- If the second condition is met, this is considered to be the first tropopause, and the loop is terminated.

It is noted that this method only identifies the *first tropopause*, and thus does not consider any tropopause folding that results in the ``leaves'' discussed above (American Meteorological Society, 2020).

## A Second (More Recent) Implementation of Calculating Tropopause Height

See Appendix A of:

Birner, T., 2006: Fine-scale structure of the extratropical tropopause region. J. Geophys. Res., 111, D04104, doi:10.1029/2005JD006301.

Here, Birner uses the same WMO (1992) definition of tropopause height. However, it is stated that with much better vertical resolution in our soundings today, the implementation of finding the tropopause height should be adjusted.

-- The temperature gradient is calculated using a two point centered finite difference for all heights

-- The temperature gradient for each height is then tested for when it is below some threshold lapse rate γTP (some value between 1Kkm and 3Kkm)

-- If this threshold is achieved, several secondary conditions are checked:

1) Is the mean of the lapse rates for two points above, and including, this height greater than or equal to γTP ?

2) Is the mean of the lapse rates for three points below this height less than γTP ?

-- If these two conditions are met, we do two more checks to finally decide the tropopause height:

1) If the mean of the lapse rate of the two points above, and including this height is strictly positive, then the tropopause height is decided to be the height at which the temperature is lowest for the six points discussed above.

2) If the mean of the lapse rate of the two points above, and including this height is 0 or negative, then two linear fits are made between the set of points above and below the height, and the tropopause height is determined to be the interesection of these two lines.

### References 


American Meteorological Society, cited 2020: "Tropopause". Glossary of Meteorology. [Available online at https://glossary.ametsoc.org/wiki/Tropopause ]

American Meteorological Society, cited 2020: "Multiple Tropopause". Glossary of Meteorology. [Available online at https://glossary.ametsoc.org/wiki/Multiple_tropopause ]

Añil J., Antuña-Marrero J., de la Torre Ramos L. and Castanheira J. (2008), Climatological features of global multiple tropopause events, J. Geophys. Res.-Atmos, 113(7):0-8.

Bethan S., Vaughan G. and Reid, S. J. (1996), A comparison of ozone and thermal tropopause heights and the impact of tropopause definition on quantifying the ozone content of the troposphere, Q. J. Royal Meteorological Society, 122: 929-944.

Birner, T., 2006: Fine-scale structure of the extratropical tropopause region. J. Geophys. Res., 111, D04104, doi:10.1029/2005JD006301.

Homeyer, C. R., Bowman, K. P., Pan, L. L., Atlas, E. L., Gao, R.‐S., and Campos, T. L. (2011), Dynamical and chemical characteristics of tropospheric intrusions observed during START08, J. Geophys. Res., 116, D06111.

Homeyer C. R. and Bowman K. P. (2013), Rossby Wave Breaking and Transport between the Tropics and Extratropics above the Subtropical Jet, J. Atmos. Sci., 70 (2): 607-626.

Manney G. L. and Hegglin M. I. (2018), Seasonal and Regional Variations of Long-Term Changes in Upper-Tropospheric Jets from Reanalyses, J. Climate, 31 (1): 423-448.

Mote, P. W., Rosenlof, K. H., McIntyre, M. E., Carr, E. S., Gille, J. C., Holton, J. R., Kinnersley, J. S., Pumphrey, H. C., Russell, J. M., and Waters, J. W. (1996), An atmospheric tape recorder: The imprint of tropical tropopause temperatures on stratospheric water vapor, J. Geophys. Res., 101( D2), 3989– 4006.

Pan L. L. and Munchak, L. A. (2011), Relationship to cloud top to the tropopause  and jet structure from CALIPSO data, Aerosol and Clouds,  J. Geophys, Res. 116.

Santer B.D., Wehner M. F., Wigley T. M. L. Sausen R., Meehl G. A., Taylor K. E., Ammann C., Arblaster J. (2003), Contributions of Anthropogenic and Natural Forcing to Recent Tropopause Height Changes, Science, 301 (5632): 479-483.


World Meteorological Organization (1992), International meteorological vocabulary, 2nd Edition, WMO, 182, 784pp. 

Xian, T. and Homeyer, C. R. (2019), Global tropopause altitudes in radiosondes and reanalyses, Atmos. Chem. Phys., 19, 5661–5678.

### ECB comments

Thanks for the really nice overview of the theory, extant definitions, and practical approaches for performing the calculation. I think the work plan you identified is good.

I like the idea to use soundings to provide an an objecive characterization of the relative errors in the different implmentations.
- Characterize both the mean absolute error as well as its variability - low sensitivity is just as important so as to not catch folks by surprise.

When converting from Fortran, look for opportunities to use existing calcualtions in MetPy (e.g., for Lapse Rate), and/or compare them to a direct finite difference which you might choose to implement.
- Note that, outside the boundaries, you can vectorize finite differencing with an array for all points not on the edge. This avoids a slow for loop. The code below gives an example of a 4th-order accurate centered finite difference for one-dimensiontal data. (Roberto, the poisson solver I sent you has a multi-dimensinonal version of this same function). 


In [ ]:
# 1D equivalent for output gradient array and input data f.
out[2:-2] = (f[:4] - 8*f[1:-3] + 8*f[3:-1] - f[4:])/12.0
